In [52]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [53]:
df=pd.read_csv('Datasets/kindle Review/all_kindle_review.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [54]:
df=df[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [55]:
# any mising
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [56]:
df['rating']=df['rating'].apply(lambda x:0 if x<3 else 1)

In [57]:
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
all_common_words=set(stopwords.words('english'))

In [58]:
corpus=[]
lemmatizer=WordNetLemmatizer()

for sen in df['reviewText']:
	sen=sen.lower()
	sen=re.sub('[^a-z A-z 0-9-]+','',sen)
	words=word_tokenize(sen)
	words=[word for word in words if word not in all_common_words]
	words=[lemmatizer.lemmatize(word) for word in words ]

	sen=" ".join(words)
	corpus.append(sen)

In [59]:
# No null values

value_size_is_null=[len(sen)>0 for sen in corpus]
value_size_is_null.count(0)

0

In [60]:
X=corpus
Y=df['rating']

In [61]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.22,random_state=42)


In [20]:
from gensim import  downloader
wv=downloader.load('word2vec-google-news-300')

In [62]:
def avg_word_vec(sen : str):
	words=word_tokenize(sen)
	words=[word for word in words if word in wv]
	vectors=[]
	for word  in words:
		vectors.append(wv[word])

	return np.mean(vectors,axis=0)


def convert_sen_to_vec(doc):
	W2V=[]
	for sen in doc:
		W2V.append(avg_word_vec(sen))

	return pd.DataFrame(W2V)

In [64]:
X_train=convert_sen_to_vec(X_train)
X_test=convert_sen_to_vec(X_test)

In [65]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.045542,0.027035,0.041632,0.092636,-0.040175,0.053072,0.077662,-0.094948,0.121221,0.060669,...,-0.021560,-0.022536,-0.145243,0.001323,-0.003248,-0.036427,0.021451,-0.092917,-0.001205,0.003551
1,0.058184,0.014621,-0.003965,0.085201,-0.081914,0.019917,0.066811,-0.040394,0.097750,0.039807,...,-0.064415,0.047426,-0.045914,0.030439,-0.083819,-0.008747,-0.017334,-0.041072,-0.003574,0.024533
2,0.056541,0.037930,-0.001986,0.084921,-0.047941,0.006700,0.088472,-0.063519,0.077387,0.053567,...,-0.057998,0.037100,-0.087931,-0.017533,-0.071017,-0.039638,0.020287,-0.063260,0.013451,-0.005055
3,0.062495,-0.081006,0.028662,0.121777,-0.058447,0.066943,0.072998,0.082715,0.010937,0.071582,...,0.047681,-0.067413,-0.090283,-0.079883,-0.014575,-0.177051,-0.063708,0.005420,0.008984,-0.014899
4,0.029961,0.010707,-0.015831,0.109202,-0.067820,0.050720,0.125931,-0.014197,0.072200,0.056450,...,-0.093150,0.018560,-0.022257,0.029885,-0.023458,-0.058365,-0.005039,-0.006072,0.015650,-0.055298


In [66]:
from sklearn.ensemble import RandomForestClassifier
random=RandomForestClassifier(n_estimators=2)

In [67]:
random.fit(X_train,Y_train)

RandomForestClassifier(n_estimators=2)

In [68]:
Y_pred=random.predict(X_test)

In [69]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(Y_test,Y_pred))

0.6151515151515151


In [70]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.46      0.69      0.55       895
           1       0.79      0.57      0.66      1745

    accuracy                           0.62      2640
   macro avg       0.62      0.63      0.61      2640
weighted avg       0.67      0.62      0.63      2640

